In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv("./data/preprocessed_all.dat")
df=pd.DataFrame(data)
df.head(10)

,date,YYYY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,...,day,nshor,wshor,almo,dh,winw,wspd,wdir,nsd,ssd
0,2009-12-31,2010.0,1.0,1.0,1.0,51.0,999.0,NaN,NaN,2.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-01,2010.0,1.0,1.0,18.0,51.0,999.0,NaN,NaN,2.28,...,1.0,6.0,4.0,2.0,3.0,4.0,3.0,2.0,16.0,13
2,2010-01-02,2010.0,1.0,3.0,23.0,51.0,999.0,NaN,NaN,1.96,...,2.0,7.0,4.0,2.0,2.0,3.0,2.0,17.0,14.0,999
3,2010-01-03,2010.0,1.0,3.0,18.0,51.0,999.0,NaN,NaN,1.84,...,3.0,6.0,4.0,2.0,2.0,2.0,4.0,11.0,15.0,999
4,2010-01-04,2010.0,1.0,4.0,17.0,51.0,999.0,NaN,NaN,2.15,...,4.0,6.0,3.0,2.0,2.0,2.0,4.0,11.0,14.0,999
5,2010-01-05,2010.0,1.0,6.0,1.0,21.0,999.0,NaN,NaN,4.20,...,5.0,18.0,6.0,2.0,2.0,2.0,4.0,11.0,14.0,999
6,2010-01-06,2010.0,1.0,6.0,17.0,51.0,999.0,NaN,NaN,3.47,...,6.0,15.0,4.0,2.0,2.0,2.0,3.0,13.0,14.0,999
7,2010-01-07,2010.0,1.0,7.0,17.0,21.0,999.0,NaN,NaN,3.99,...,7.0,15.0,8.0,3.0,4.0,2.0,3.0,2.0,14.0,999
8,2010-01-08,2010.0,1.0,8.0,17.0,21.0,999.0,NaN,NaN,3.51,...,8.0,15.0,6.0,3.0,3.0,2.0,2.0,17.0,15.0,999
9,2010-01-09,2010.0,1.0,9.0,17.0,21.0,999.0,NaN,NaN,2.46,...,9.0,10.0,5.0,2.0,2.0,4.0,4.0,11.0,16.0,999


In [2]:
# df.nshor=np.where(df.nshor.isnull(),df['nshor'].mean(),df['nshor'])
# df.WVHT=np.where(df.WVHT.isnull(),df['WVHT'].mean(),df['WVHT'])
from math import sqrt
from math import pi
#since we have only 2 variables hence instead of handling nulls by replacing them with mean / median etc we are removing null observations 
df = df[df['nshor'].notna()]
df = df[df['WVHT'].notna()]
g=9.8
df["kr"]=""
df["h_br"]=""
df["h_surf"]=""
#calculting breaker height
df["h_br"]=pow(df["WVHT"], (4/5))*pow((1/sqrt(9.8))*(g*df["DPD"]/(4*pi)),(2/5))
#calculating refrection coeffiecient
df["kr"]=-.003*pow(df["h_br"],3)+.0099*pow(df["h_br"],2)-0.025*df["h_br"]+1.0747
#estimated surf height
df["h_surf"]=df["h_br"]*df["kr"]
#converting surf height to feet 
df["h_surf"]=df["h_surf"]*3.28
#half actual face height
df["h_surf"]=df["h_surf"]/2

In [3]:
df["h_surf"][0:10]

1     4.822202
2     4.997955
3     4.527979
4     4.626003
5     6.736800
6     6.693369
7     6.744664
8     6.584453
9     5.397040
10    5.489331
Name: h_surf, dtype: float64

In [4]:
df["nshor"][0:10]

1      6.0
2      7.0
3      6.0
4      6.0
5     18.0
6     15.0
7     15.0
8     15.0
9     10.0
10     7.0
Name: nshor, dtype: float64

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mae_xgb = mean_absolute_error(df['nshor'],df['h_surf'])
print("MAE: ", mae_xgb)
m_s_e=mean_squared_error(df["nshor"],df["h_surf"])
print("RMSE",np.sqrt(m_s_e))

MAE:  1.949323960929721
RMSE 3.5501823095209577
